In [4]:
!pip install proxy

In [1]:
import requests
from bs4 import BeautifulSoup
# from proxy import ProxyAgent
import pandas as pd
import itertools
import re
from tqdm import tqdm_notebook
import string

# Парсинг ссылок на курсы

In [6]:
urls = ["https://www.coursera.org/browse/computer-science?languages=en", 
        "https://www.coursera.org/browse/data-science?languages=en"]

 Тут короче я тупой и просто достал ссылки подкатегорий курсов

In [7]:
def get_subcat(url):
    req_url = requests.get(url)
    soup_url = BeautifulSoup(req_url.text, 'lxml')
    print(soup_url)
    all_subcat = soup_url.find("div", class_ ="rc-SubdomainsIndex")
    
    subcat_urls_html = all_subcat.find_all("a", class_ = "index-item-link")

    subcat_urls = [i["href"][1:] for i in subcat_urls_html]
    return subcat_urls

In [181]:
get_subcat(urls[0])

<!DOCTYPE html>
<html itemtype="http://schema.org" lang="en" xmlns:fb="http://ogp.me/ns/fb#"><head><meta content="IE=Edge,chrome=IE7" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><meta content="Coursera" property="og:site_name"/><meta content="727836538,4807654" property="fb:admins"/><meta content="823425307723964" property="fb:app_id"/><meta content="Coursera" name="twitter:site"/><meta content="Coursera" name="twitter:app:name:iphone"/><meta content="Coursera" name="twitter:app:name:ipad"/><meta content="Coursera" name="twitter:app:name:googleplay"/><meta content="id736535961" name="twitter:app:id:iphone"/><meta content="id736535961" name="twitter:app:id:ipad"/><meta content="org.coursera.android" name="twitter:app:id:googleplay"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link href="https://d3njjcbhbojbot.cloudfront.net/web/images/favicons/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/><link href="https://d3njjcbhbojbot.cloudfron

AttributeError: 'NoneType' object has no attribute 'find_all'

In [145]:
def get_courses_list(cat, subcat):
    
    url = "https://www.coursera.org/browse/{}/{}?languages=en".format(cat, subcat)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    
    pagination = soup.find_all("span", class_ = "horizontal-box align-items-absolute-center")
    num_pages = max([i.text for i in pagination if i.text.isdigit() == True])
    
    course_url = []
    
    for i in range(1, int(num_pages) + 1): 
        page = url + "&page={}".format(i)
        print(page)
        
        req_course = requests.get(page)
        soup_course = BeautifulSoup(req_course.text, "lxml")
        all_url_list = soup_course.find_all("a", class_ = "rc-OfferingCard nostyle")
        
        for a in all_url_list:
            course_url.append("https://www.coursera.org" + a["href"])
    return course_url

In [160]:
len(get_courses_list("computer-science", "design-and-product"))

https://www.coursera.org/browse/computer-science/design-and-product?languages=en&page=1
https://www.coursera.org/browse/computer-science/design-and-product?languages=en&page=2
https://www.coursera.org/browse/computer-science/design-and-product?languages=en&page=3
https://www.coursera.org/browse/computer-science/design-and-product?languages=en&page=4


77

In [146]:
all_urls = []

In [147]:
sub_cat = {"computer_science" : get_subcat(urls[0]),
      "data_science": get_subcat(urls[1])}

In [148]:
sub_cat

{'computer_science': ['software-development',
  'mobile-and-web-development',
  'algorithms',
  'computer-security-and-networks',
  'design-and-product'],
 'data_science': ['data-analysis',
  'machine-learning',
  'probability-and-statistics']}

In [149]:
for cat, sub in sub_cat.items():
    for i in sub:
        urls = get_courses_list(cat, i)
        all_urls.append(urls)

https://www.coursera.org/browse/computer_science/software-development?languages=en&page=1
https://www.coursera.org/browse/computer_science/software-development?languages=en&page=2
https://www.coursera.org/browse/computer_science/software-development?languages=en&page=3
https://www.coursera.org/browse/computer_science/software-development?languages=en&page=4
https://www.coursera.org/browse/computer_science/software-development?languages=en&page=5
https://www.coursera.org/browse/computer_science/software-development?languages=en&page=6
https://www.coursera.org/browse/computer_science/software-development?languages=en&page=7
https://www.coursera.org/browse/computer_science/software-development?languages=en&page=8
https://www.coursera.org/browse/computer_science/software-development?languages=en&page=9
https://www.coursera.org/browse/computer_science/mobile-and-web-development?languages=en&page=1
https://www.coursera.org/browse/computer_science/mobile-and-web-development?languages=en&page=

In [161]:
all_urls

[['https://www.coursera.org/specializations/data-science-python',
  'https://www.coursera.org/specializations/python',
  'https://www.coursera.org/specializations/data-structures-algorithms',
  'https://www.coursera.org/specializations/java-programming',
  'https://www.coursera.org/specializations/scala',
  'https://www.coursera.org/specializations/algorithms',
  'https://www.coursera.org/specializations/computer-fundamentals',
  'https://www.coursera.org/specializations/product-management',
  'https://www.coursera.org/specializations/full-stack-mobile-app-development',
  'https://www.coursera.org/specializations/agile-development',
  'https://www.coursera.org/specializations/iot',
  'https://www.coursera.org/specializations/java-object-oriented',
  'https://www.coursera.org/specializations/r',
  'https://www.coursera.org/specializations/cloud-computing',
  'https://www.coursera.org/specializations/software-design-architecture',
  'https://www.coursera.org/specializations/pcdp',
  'htt

In [166]:
import itertools
coursers_url = list(itertools.chain(*all_urls))

In [168]:
pd.DataFrame(coursers_url).to_csv("all_urls.csv")

In [193]:
def from_spec_to_course(all_urls):
    courses = []
    for spec in all_urls:
        if "specializations" in spec:
            req = requests.get(spec)
            soup = BeautifulSoup(req.text, "lxml")
            spec_urls = soup.find_all("a")
        
            for i in spec_urls:
                if "/learn/" in i["href"]:
                    courses.append("https://www.coursera.org" + i["href"])
    return courses

In [194]:
new_urls = from_spec_to_course(coursers_url)

In [196]:
len(new_urls)

532

In [201]:
urls_final = [i for i in coursers_url if "specializations" not in i] + new_urls

In [208]:
urls_final

['https://www.coursera.org/learn/python',
 'https://www.coursera.org/learn/algorithms-part1',
 'https://www.coursera.org/learn/python-data-analysis',
 'https://www.coursera.org/learn/algorithmic-toolbox',
 'https://www.coursera.org/learn/python-data',
 'https://www.coursera.org/learn/python-network-data',
 'https://www.coursera.org/learn/matlab',
 'https://www.coursera.org/learn/gcp-fundamentals',
 'https://www.coursera.org/learn/learn-to-program',
 'https://www.coursera.org/learn/logic-introduction',
 'https://www.coursera.org/learn/python-databases',
 'https://www.coursera.org/learn/object-oriented-java',
 'https://www.coursera.org/learn/android-app',
 'https://www.coursera.org/learn/progfun1',
 'https://www.coursera.org/learn/interactive-python-1',
 'https://www.coursera.org/learn/big-data-essentials',
 'https://www.coursera.org/learn/python-programming-introduction',
 'https://www.coursera.org/learn/algorithms-part2',
 'https://www.coursera.org/learn/r-programming-environment',
 'h

In [2]:
urls_final = pd.read_csv("all_courses_urls.csv")

In [3]:
urls_final = urls_final.drop('Unnamed: 0',axis=1)

In [4]:
urls_final

,0
0,https://www.coursera.org/learn/python
1,https://www.coursera.org/learn/algorithms-part1
2,https://www.coursera.org/learn/python-data-ana...
3,https://www.coursera.org/learn/algorithmic-too...
4,https://www.coursera.org/learn/python-data
5,https://www.coursera.org/learn/python-network-...
6,https://www.coursera.org/learn/matlab
7,https://www.coursera.org/learn/gcp-fundamentals
8,https://www.coursera.org/learn/learn-to-program
9,https://www.coursera.org/learn/logic-introduction


# Парсинг инфы с курсов

In [20]:
df_final = pd.DataFrame()

In [18]:
df_final.index

Index([], dtype='object')

In [ ]:
df_final = pd.DataFrame()
for url in tqdm_notebook(urls_final['0']):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    print(url)
    about = " ".join([i.getText(separator=' ') for i in soup.find_all("p", class_= "body-1-text course-description")]).lower()
    try:
        rows = soup.find_all("td", class_ = "td-data")
        rating = soup.find_all("td", class_ = "td-data")[-1].text.split(" ")[0]
        all_ratings = soup.find("div", class_ = "ratings-text headline-2-text").text.split(" ")[-2]
        syll = " ".join([i.getText(separator=' ') for i in soup.find_all("div", class_ = "week-body")]).lower()
    except Exception as e:
        print(e)
        
    level = None
    for row in rows:
        if row.text in ['Beginner', 'Advanced', 'Intermediate']:
            level = row.text
        
    re_rate = re.compile(r',')
    re_about = re.compile(r'about this course')
    re_syll = re.compile(r'Graded: |[0-9]{1,2} videos|[0-9]{1,2} readings')
    re_chars = re.compile(r'[^a-z ]*')
    re_punct = re.compile(r"[^\w\s-]")
    re_space = re.compile(r'[ ]{1,}')
    re_ap = re.compile(r"'\w")
    
    about = re_about.sub('', about)
    about = re_ap.sub('', about)
    about = re_punct.sub(' ', about)
    about = re_chars.sub('', about)
    about = re_space.sub(' ', about)
    
    syll = re_syll.sub(' ', syll)
    syll = re_ap.sub('', syll)
    syll = re_punct.sub(' ', syll)
    syll = re_chars.sub('', syll)
    syll = re_space.sub(' ', syll)
    
    all_ratings = re_rate.sub('', all_ratings)
    
    data = [{"index": url,
            "level": level,
           "rating": rating,
           "rating_count": all_ratings,
           "syllabus": syll,
           "about": about}]

    data = pd.DataFrame(data)
    data.set_index('index', inplace=True)
    df_final = df_final.append(data)
#     print(data)


A Jupyter Widget

https://www.coursera.org/learn/python
https://www.coursera.org/learn/algorithms-part1
https://www.coursera.org/learn/python-data-analysis
https://www.coursera.org/learn/algorithmic-toolbox
https://www.coursera.org/learn/python-data
https://www.coursera.org/learn/python-network-data
https://www.coursera.org/learn/matlab
https://www.coursera.org/learn/gcp-fundamentals
https://www.coursera.org/learn/learn-to-program
https://www.coursera.org/learn/logic-introduction
https://www.coursera.org/learn/python-databases
https://www.coursera.org/learn/object-oriented-java
https://www.coursera.org/learn/android-app
https://www.coursera.org/learn/progfun1
https://www.coursera.org/learn/interactive-python-1
https://www.coursera.org/learn/big-data-essentials
https://www.coursera.org/learn/python-programming-introduction
https://www.coursera.org/learn/algorithms-part2
https://www.coursera.org/learn/r-programming-environment
https://www.coursera.org/learn/python-text-mining
https://www.coursera.org/lear

In [33]:
df_final.head(10)

,about,level,rating,rating_count,syllabus
index,,,,,
https://www.coursera.org/learn/python,about this course this course aims to teach ev...,None,4.8,29617,chapter one why we program these are the cours...
https://www.coursera.org/learn/algorithms-part1,about this course this course covers the essen...,Intermediate,4.9,2648,course introduction welcome to algorithms part...
https://www.coursera.org/learn/python-data-analysis,about this course this course will introduce t...,Intermediate,4.5,3835,week in this week youl get an introduction to ...
https://www.coursera.org/learn/algorithmic-toolbox,about this course the course covers basic algo...,None,4.6,2354,welcome welcome to the first module of data st...
https://www.coursera.org/learn/python-data,about this course this course will introduce t...,None,4.8,16113,chapter six strings in this class we pick up w...
https://www.coursera.org/learn/python-network-data,about this course this course will show how on...,None,4.8,11049,getting started in this section you will insta...
https://www.coursera.org/learn/matlab,about this course this course teaches computer...,Beginner,4.8,1958,course pages video expand video course preview...
https://www.coursera.org/learn/gcp-fundamentals,about this course attention learners intereste...,Intermediate,4.6,1860,welcome to gcp fundamentals core infrastructur...
https://www.coursera.org/learn/learn-to-program,about this course behind every mouse click and...,Beginner,4.7,1262,python variables and functions this module giv...


In [34]:
df_final.to_csv('./coursera.csv')